<a href="https://colab.research.google.com/github/XinTian-1101/Data-Mining/blob/main/Data_Mining_Individual_Assigment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Data Mining Individual Assignment : Automated Feature Engineering:

In [ ]:
!pip install featuretools
!pip install faker

### **1 Import libraries**

In [ ]:
import featuretools as ft
from featuretools.primitives import AggregationPrimitive, TransformPrimitive
import numpy as np
import pandas as pd
import random
from faker import Faker
from sklearn.preprocessing import StandardScaler, LabelEncoder

### **2. Random Generate Data Based On Constraint Set**

In [ ]:
# Generate Realistic Fake Data
fake = Faker()

In [ ]:
np.random.seed(42)
num_orders = 50
num_customers = 45
num_products = 4

#### 2.1 Product Data

In [ ]:
# Generate Product Data
product_data = {
    'ProductID':range(201,201+num_products),
    'Name':['Laptop', 'Tablet','Smartphone','Gaming Chair'],
    'Category':['Electronic','Electronic','Electronic','Home'],
    'Price' : [3600,2900,3450,750]
}

product_df = pd.DataFrame(product_data)
print("Product Data\n",product_df.head())

Product Data
    ProductID          Name    Category  Price
0        201        Laptop  Electronic   3600
1        202        Tablet  Electronic   2900
2        203    Smartphone  Electronic   3450
3        204  Gaming Chair        Home    750


#### 2.2 Customer Data

In [ ]:
# Generate Customer Data
customers_data = {
    'CustomerID':range(101,101+num_customers)
}

In [ ]:
# Set the earliest possible signup date and the latest possible date to generate orders
earliest_signup_date = pd.to_datetime("2024-06-01")
latest_order_date = pd.to_datetime("2024-10-31")

In [ ]:
names = [fake.name() for _ in range(num_customers)]
emails =  [f"{name.lower().replace(' ','')}@example.com" for name in names]
signup_dates = [fake.date_between_dates(date_start = earliest_signup_date, date_end = latest_order_date) for _ in range (num_customers)]

# Assign generated data to customer_data dictionery
customers_data['Name'] = names
customers_data['Email'] = emails
customers_data['Income'] = np.random.randint(1500,9000,num_customers)
customers_data['SignupDate'] = signup_dates


customer_df = pd.DataFrame(customers_data)
print("Customer Data \n", customer_df.head())

Customer Data 
    CustomerID            Name                      Email  Income  SignupDate
0         101  Jeanette Brady  jeanettebrady@example.com    8770  2024-08-19
1         102  Micheal Thomas  michealthomas@example.com    2360  2024-07-19
2         103  Timothy Larson  timothylarson@example.com    6890  2024-09-21
3         104    Kelly Chaney    kellychaney@example.com    6726  2024-09-03
4         105    David Wright    davidwright@example.com    6691  2024-08-15


#### 2.3 Order Data

In [ ]:
# Generate Order Data
order_data = {
    'OrderID' : range(301,301+num_orders),
    'CustomerID': np.random.choice(customer_df['CustomerID'], num_orders), # Allowing a customer to repeat orders
}

order_dates = []
ship_dates = []

for customer_id in order_data['CustomerID']:
    customer_signup_date = customer_df.loc[customer_df['CustomerID'] == customer_id, 'SignupDate'].values[0]

    # Generate order dates after or on the signupdate
    order_date = fake.date_between_dates(date_start = customer_signup_date, date_end = latest_order_date)
    order_dates.append(order_date)

    # Generate ShipDate on or after OrderDate, or None for undelivered orders
    if random.random() < 0.95 : # 95% chance for delivery
        ship_date = fake.date_between_dates(date_start = order_date, date_end=latest_order_date)
    else :
        ship_date = None # 0.05% chance for undelivered
    ship_dates.append(ship_date)

order_data['OrderDate'] = order_dates
order_data['ShipDate'] = ship_dates

order_df = pd.DataFrame(order_data)

print("Order Data \n",order_df.head())
print("\n Customer Data \n", customer_df.head())

Order Data 
    OrderID  CustomerID   OrderDate    ShipDate
0      301         144  2024-10-14  2024-10-21
1      302         103  2024-10-21  2024-10-29
2      303         137  2024-08-31  2024-09-13
3      304         107  2024-10-27        None
4      305         121  2024-07-24  2024-10-17

 Customer Data 
    CustomerID            Name                      Email  Income  SignupDate
0         101  Jeanette Brady  jeanettebrady@example.com    8770  2024-08-19
1         102  Micheal Thomas  michealthomas@example.com    2360  2024-07-19
2         103  Timothy Larson  timothylarson@example.com    6890  2024-09-21
3         104    Kelly Chaney    kellychaney@example.com    6726  2024-09-03
4         105    David Wright    davidwright@example.com    6691  2024-08-15


In [ ]:
# Filter the customers who are in the OrderData to verify constraints
filtered_customer_df = customer_df[customer_df['CustomerID'].isin(order_df['CustomerID'])]
merged_df = pd.merge(order_df, customer_df, on="CustomerID", how="inner")

print("Order Data (Sampled to Verify Constraints):\n", merged_df[['OrderID', 'CustomerID', 'OrderDate', 'ShipDate']].head())
print("\nCustomer Data Related to Orders:\n", merged_df[['CustomerID', 'Name', 'Email', 'SignupDate','Income']].head())

Order Data (Sampled to Verify Constraints):
    OrderID  CustomerID   OrderDate    ShipDate
0      301         144  2024-10-14  2024-10-21
1      302         103  2024-10-21  2024-10-29
2      303         137  2024-08-31  2024-09-13
3      304         107  2024-10-27        None
4      305         121  2024-07-24  2024-10-17

Customer Data Related to Orders:
    CustomerID            Name                      Email  SignupDate  Income
0         144   Jamie Alvarez   jamiealvarez@example.com  2024-07-30    8601
1         103  Timothy Larson  timothylarson@example.com  2024-09-21    6890
2         137    Steven Moore    stevenmoore@example.com  2024-08-20    2582
3         107  Robert Stevens  robertstevens@example.com  2024-10-25    4592
4         121  Wayne Erickson  wayneerickson@example.com  2024-07-23    2269


#### 2.4 OrderDetails Data

In [ ]:
# Initialize empty lists for order details columns
order_ids = []
product_ids = []
quantities = []
discounts = []

product_discounts = {
    201 : 0.0,
    202 : 0.1,
    203 : 0.0,
    204 : 0.05
}

min_item_per_order = 1
max_item_per_order = 3

for order_id in order_df['OrderID']:
    num_items = np.random.randint(min_item_per_order,max_item_per_order+1)
    selected_product_ids = np.random.choice(product_df['ProductID'],num_items,replace=True)

    # For each selected product, generate a quantity and discount
    for product_id in selected_product_ids:
        order_ids.append(order_id) # Same order_id for all items in this order
        product_ids.append(product_id)
        quantities.append(np.random.randint(1,3))
        discounts.append(product_discounts[product_id])

    order_detail_df = pd.DataFrame({
    'OrderID':order_ids,
    'ProductID':product_ids,
    'Quantity':quantities,
    'Discount': discounts
    })

print("Order Details Data \n", order_detail_df.head())

Order Details Data 
    OrderID  ProductID  Quantity  Discount
0      301        201         1      0.00
1      302        201         1      0.00
2      302        201         1      0.00
3      302        201         2      0.00
4      303        204         2      0.05


In [ ]:
# Check total orders and total customers who placed orders
total_orders = order_df['OrderID'].nunique()
total_customers_placed_order = order_df['CustomerID'].nunique()

print("Total Orders:", total_orders)
print("Total Customers who Placed Orders:", total_customers_placed_order)

Total Orders: 50
Total Customers who Placed Orders: 31


In [ ]:
# Check for order_details_df got same amount as order_df
unique_orderID_in_orderDetails = order_detail_df['OrderID'].nunique()

# Total Registered Customers
registered_customers = customer_df['CustomerID'].nunique()

# Total products sell
products = product_df['ProductID'].nunique()

# Check for order_details_df got same amount as product_df
unique_productID_in_orderDetails = order_detail_df['ProductID'].nunique()

# Display the unique counts
print("Unique Order IDs in order_details_df:", unique_orderID_in_orderDetails)
print("Registered Customers:", registered_customers)
print("Products:", products)
print("Unique Product IDs in order_detail_df:", unique_productID_in_orderDetails)

Unique Order IDs in order_details_df: 50
Registered Customers: 45
Products: 4
Unique Product IDs in order_detail_df: 4


### **3. Save Generated Data**

In [ ]:
order_df.to_csv('order_data.csv',index=False)
order_detail_df.to_csv('order_details_data.csv', index=False)
product_df.to_csv('product_data.csv', index=False)
customer_df.to_csv('customer_data.csv', index=False)

### **4. Define Entities & Relationships**

In [ ]:
#Initiaze Entity Set
es = ft.EntitySet(id = 'ecommerce_set')

In [ ]:
# Create Empty Entity Set
#ORDERS
orders_entity = es.add_dataframe(
    dataframe_name = 'Orders',
    dataframe = order_df,
    index = 'OrderID',
    time_index='OrderDate')

#ORDER DETAILS
order_details_entity = es.add_dataframe(
    dataframe_name = 'OrderDetails',
    dataframe = order_detail_df,
    make_index=True,
    index = 'OrderDetailsID')

#PRODUCTS
products_entity = es.add_dataframe(
    dataframe_name = 'Products',
    dataframe = product_df,
    index = 'ProductID')

#CUSTOMER
customers_entity = es.add_dataframe(
    dataframe_name="Customers",
    dataframe=customer_df,
    index="CustomerID",
    time_index="SignupDate"
)

/usr/local/lib/python3.10/dist-packages/woodwork/type_sys/utils.py:33: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(
/usr/local/lib/python3.10/dist-packages/woodwork/type_sys/utils.py:33: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(
/usr/local/lib/python3.10/dist-packages/woodwork/type_sys/utils.py:33: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(
/usr/local/lib/python3.10/dist-packages/woodwork/type_sys/utils.py:33: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. 

In [ ]:
# Define Relationships
customer_order_relationship = es.add_relationship(
    parent_dataframe_name="Customers",
    parent_column_name="CustomerID",
    child_dataframe_name="Orders",
    child_column_name="CustomerID"
)

order_orderdetails_relationship = es.add_relationship(
    parent_dataframe_name="Orders",
    parent_column_name="OrderID",
    child_dataframe_name="OrderDetails",
    child_column_name="OrderID"
)

product_orderdetails_relationship = es.add_relationship(
    parent_dataframe_name="Products",
    parent_column_name="ProductID",
    child_dataframe_name="OrderDetails",
    child_column_name="ProductID"
)

In [ ]:
# PRINT THE ENTITY SET
print(es)

Entityset: ecommerce_set
  DataFrames:
    Orders [Rows: 50, Columns: 4]
    OrderDetails [Rows: 92, Columns: 5]
    Products [Rows: 4, Columns: 4]
    Customers [Rows: 45, Columns: 5]
  Relationships:
    Orders.CustomerID -> Customers.CustomerID
    OrderDetails.OrderID -> Orders.OrderID
    OrderDetails.ProductID -> Products.ProductID


In [ ]:
# SET PANDAS DISPLAY OPTIONS TO SHOW ALL COLUMNS AND ROWS
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

### **5. Check Any Missing Values**

In [ ]:
print("Missing values in order_df : \n", order_df.isnull().sum())
print("\nMissing values in order_details_df:\n", order_detail_df.isnull().sum())
print("\nMissing values in product_df:\n", product_df.isnull().sum())
print("\nMissing values in customer_df:\n", customer_df.isnull().sum())

Missing values in order_df : 
 OrderID       0
CustomerID    0
OrderDate     0
ShipDate      3
dtype: int64

Missing values in order_details_df:
 OrderDetailsID    0
OrderID           0
ProductID         0
Quantity          0
Discount          0
dtype: int64

Missing values in product_df:
 ProductID    0
Name         0
Category     0
Price        0
dtype: int64

Missing values in customer_df:
 CustomerID    0
Name          0
Email         0
Income        0
SignupDate    0
dtype: int64


Although ShipDate has missing values but since it represents orders that haven’t been shipped yet so instead of removing them, we still keeps this information.

### **6. Feature Syntesis for Derived Field**


In [ ]:
# Check actual columns of order_details_df
print(order_detail_df.columns)

Index(['OrderDetailsID', 'OrderID', 'ProductID', 'Quantity', 'Discount'], dtype='object')


In [ ]:
agg_primitives = ["mean", "sum", "count"]
trans_primitives = ["month", "day","year"]

features_order_details, feature_defs_order_details = ft.dfs(
    entityset=es,
    target_dataframe_name="OrderDetails",
    agg_primitives=agg_primitives,
    trans_primitives=trans_primitives,
    max_depth=2
)

print("Generated Features for OrderDetails:")
print(features_order_details.head())

/usr/local/lib/python3.10/dist-packages/featuretools/computational_backends/feature_set_calculator.py:785: FutureWarning: The provided callable <function sum at 0x784c7a9000d0> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  ).agg(to_agg)
/usr/local/lib/python3.10/dist-packages/featuretools/computational_backends/feature_set_calculator.py:785: FutureWarning: The provided callable <function mean at 0x784c7a901000> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  ).agg(to_agg)
/usr/local/lib/python3.10/dist-packages/featuretools/computational_backends/feature_set_calculator.py:785: FutureWarning: The provided callable <function mean at 0x784c7a901000> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable w

Generated Features for OrderDetails:
                OrderID  ProductID  Quantity  Discount  Orders.CustomerID  \
OrderDetailsID                                                              
0                   301        201         1      0.00                144   
1                   302        201         1      0.00                103   
2                   302        201         1      0.00                103   
3                   302        201         2      0.00                103   
4                   303        204         2      0.05                137   

                Products.Price  Orders.COUNT(OrderDetails)  \
OrderDetailsID                                               
0                         3600                           1   
1                         3600                           3   
2                         3600                           3   
3                         3600                           3   
4                          750                     

#### 6.1 Line Item Revenue

In [ ]:
# Calculate LineItemRevenue
features_order_details["LineItemRevenue"] = (
    features_order_details["Quantity"] *
    features_order_details["Products.Price"] *
    (1 - features_order_details["Discount"])
)

features_order_details["LineItemRevenue"] = features_order_details["LineItemRevenue"].round(0).astype(int)
print(features_order_details [["OrderID","LineItemRevenue"]].head())

                OrderID  LineItemRevenue
OrderDetailsID                          
0                   301             3600
1                   302             3600
2                   302             3600
3                   302             7200
4                   303             1425


In [ ]:
order_revenue_summary = features_order_details.groupby('OrderID')['LineItemRevenue'].sum().reset_index()
order_revenue_summary.columns = ['OrderID', 'TotalLineItemRevenue']
print(order_revenue_summary.head())

   OrderID  TotalLineItemRevenue
0      301                  3600
1      302                 14400
2      303                  1425
3      304                 13950
4      305                  3450


In [ ]:
order_quantity_summary = features_order_details.groupby('OrderID')['Quantity'].sum().reset_index()
order_quantity_summary.columns = ['OrderID', 'TotalQuantityOrdered']

order_df = order_df.merge(order_quantity_summary, on="OrderID", how="left")
order_df = order_df.merge(order_revenue_summary, on="OrderID", how="left")

order_df["TotalQuantityOrdered"] = order_df["TotalQuantityOrdered"].fillna(0).astype(int)
order_df["TotalLineItemRevenue"] = order_df["TotalLineItemRevenue"].fillna(0)

order_df = order_df.sort_values(by="OrderID").reset_index(drop=True)
print("Updated Order Data:\n", order_df)

Updated Order Data:
     OrderID  CustomerID  OrderDate   ShipDate  TotalQuantityOrdered  \
0       301         144 2024-10-14 2024-10-21                     1   
1       302         103 2024-10-21 2024-10-29                     4   
2       303         137 2024-08-31 2024-09-13                     2   
3       304         107 2024-10-27        NaT                     4   
4       305         121 2024-07-24 2024-10-17                     1   
5       306         109 2024-10-13 2024-10-27                     3   
6       307         139 2024-09-21 2024-10-29                     5   
7       308         118 2024-09-11 2024-10-05                     4   
8       309         104 2024-09-03 2024-10-04                     2   
9       310         125 2024-08-15 2024-10-04                     2   
10      311         114 2024-10-30 2024-10-30                     4   
11      312         109 2024-10-21 2024-10-21                     5   
12      313         126 2024-08-14 2024-09-15           

#### 6.2 Total Revenue Per Product

In [ ]:
# Group
TotalRevenuePerProduct = features_order_details.groupby("ProductID")["LineItemRevenue"].sum().reset_index()
TotalRevenuePerProduct.columns = ["ProductID", "TotalRevenuePerProduct"]
print(TotalRevenuePerProduct.head())

   ProductID  TotalRevenuePerProduct
0        201                  129600
1        202                   57420
2        203                  134550
3        204                   27781


In [ ]:
product_df = product_df.merge(TotalRevenuePerProduct, on='ProductID', how='left')
product_df['TotalRevenuePerProduct'] = product_df['TotalRevenuePerProduct'].fillna(0)
print("Product Data\n", product_df.head())

Product Data
    ProductID          Name    Category  Price  TotalRevenuePerProduct
0        201        Laptop  Electronic   3600                  129600
1        202        Tablet  Electronic   2900                   57420
2        203    Smartphone  Electronic   3450                  134550
3        204  Gaming Chair        Home    750                   27781


#### 6.3 Total Customers who Placed Orders

In [ ]:
TotalCustomersPlacedOrders= features_order_details["Orders.CustomerID"].nunique()

print("Total Customers who Placed Order:", TotalCustomersPlacedOrders)

Total Customers who Placed Order: 31


#### 6.4 Total Registered Customers

In [ ]:
TotalRegisteredCustomers = customer_df["CustomerID"].nunique()

print("Total Registered Customers:", TotalRegisteredCustomers)

Total Registered Customers: 45


#### 6.5 Total Orders

In [ ]:
TotalOrders = features_order_details["OrderID"].nunique()

print("Total Orders:", TotalOrders)

Total Orders: 50


#### 6.6 Average Order Value (AOV)

In [ ]:
TotalRevenues = features_order_details["LineItemRevenue"].sum()
print("Total Revenues:", int(TotalRevenues))

Total Revenues: 349351


In [ ]:
AOV = TotalRevenues / TotalOrders
print("Average Order Value (AOV):", int(AOV))

Average Order Value (AOV): 6987


#### 6.7 Average Purchase Frequency (APF)

In [ ]:
APF = TotalOrders / TotalCustomersPlacedOrders
APF = round(APF, 2)
print("Average Purchase Frequency (APF):", APF)

Average Purchase Frequency (APF): 1.61


#### 6.8 Customer Lifetime Value (CLV)

In [ ]:
# From June 2024 untill Octorber 2024 : 153 days / 365 days ≈ 0.42 years
average_customer_lifetime_years = 0.42

CLV = AOV * APF * average_customer_lifetime_years
CLV = round(CLV, 2)
print("Customer Lifetime Value (CLV):", CLV)

Customer Lifetime Value (CLV): 4724.62


#### 6.9 Customer Segmentation and Profiling

In [ ]:
bins = [0, 2499, 3169, 3969, 4849, 5879, 7099, 8699, 10959]
labels = ['B1', 'B2', 'B3', 'B4', 'M1', 'M2', 'M3', 'M4']

customer_df['IncomeLevel'] = pd.cut(customer_df['Income'], bins=bins, labels=labels)

In [ ]:
average_income = round(customer_df['Income'].mean(), 2)
avg_income_per_level = customer_df.groupby('IncomeLevel')['Income'].mean().round(2)

print("Average Income:\n", average_income)
print("\nAverage Income per Level:\n", avg_income_per_level)

Average Income:
 5226.49

Average Income per Level:
 IncomeLevel
B1    2051.86
B2    2633.00
B3    3572.40
B4    4375.00
M1    5179.50
M2    6590.50
M3    7838.38
M4    8873.00
Name: Income, dtype: float64


<ipython-input-190-b50574e904f0>:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  avg_income_per_level = customer_df.groupby('IncomeLevel')['Income'].mean().round(2)


In [ ]:
# Revenue by Income Group
customer_orders = order_df.merge(customer_df[['CustomerID', 'Income', 'IncomeLevel']], on='CustomerID', how='left')

orders_per_income_group = customer_orders.groupby('IncomeLevel')['OrderID'].count().reset_index()
orders_per_income_group.columns = ['IncomeLevel', 'TotalOrders']

customer_order_revenue = features_order_details.merge(customer_orders[['OrderID', 'IncomeLevel']], on='OrderID', how='left')

revenue_per_income_group = customer_order_revenue.groupby('IncomeLevel')['LineItemRevenue'].sum().reset_index()
revenue_per_income_group.columns = ['IncomeLevel', 'TotalRevenue']

income_group_insights = orders_per_income_group.merge(revenue_per_income_group, on='IncomeLevel', how='left')
print(income_group_insights)

  IncomeLevel  TotalOrders  TotalRevenue
0          B1           10         64993
1          B2            2          4035
2          B3            4         20271
3          B4            8         60441
4          M1            3         23287
5          M2            8         51944
6          M3           14        120930
7          M4            1          3450


<ipython-input-191-14a0bdd75051>:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  orders_per_income_group = customer_orders.groupby('IncomeLevel')['OrderID'].count().reset_index()
<ipython-input-191-14a0bdd75051>:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  revenue_per_income_group = customer_order_revenue.groupby('IncomeLevel')['LineItemRevenue'].sum().reset_index()


### **7. Save New Dataset with Generated Features**

In [ ]:
order_df.to_csv('order_data.csv',index=False)
order_detail_df.to_csv('order_details_data.csv', index=False)
product_df.to_csv('product_data.csv', index=False)
customer_df.to_csv('customer_data.csv', index=False)